In [1]:
import json
import heapq
from collections import defaultdict

# Load the input data (replace with actual file path or data)
with open('../Data/10_computable_moments.txt', 'r') as file:
    content = file.read()

# Parse the input data
lines = content.split('\n')[1:]  # Skip header line
Final_List = []
for idx, line in enumerate(lines):
    if line:
        parts = line.split()
        Final_List.append({
            "id": [idx],
            "type": parts[0],
            "tagLength": int(parts[1]),
            "tags": parts[2:]
        })
        

In [2]:
Portrait_Images = [img for img in Final_List if img["type"] == "P"]
Landscape_Images = [img for img in Final_List if img["type"] == "L"]


In [3]:
def min_form_number_of_common_tags(row1, row2):
    set1, set2 = row1["tags"], row2["tags"]
    return min(len(set1 - set2), len(set2 - set1), len(set1 & set2))

In [4]:
Frame_Portrait_Images = []

if(len(Portrait_Images)>0):
    
    Temp_Portrait_Images = Portrait_Images.copy()
    
    for i in range(len(Portrait_Images)):
        if len(Temp_Portrait_Images) == 0:
            break
        
        element1 = Temp_Portrait_Images.pop(0)
        
        for j in range(len(Temp_Portrait_Images)):
            element2 = Temp_Portrait_Images[j]
            common_tags = len(set(element1.get('tags')) & set(element2.get('tags')))
            if common_tags < 1:
                combined_tags = list(set(element1.get('tags')) | set(element2.get('tags')))
                Frame_Portrait_Images.append({
                    "id": [element1['id'][0], element2['id'][0]],
                    "type": "P",
                    "tagLength": len(combined_tags),
                    "tags": combined_tags
                })
                Temp_Portrait_Images.pop(j)
                break
            else :
                if j == len(Temp_Portrait_Images) - 1:
                    combined_tags = list(set(element1.get('tags')) | set(element2.get('tags')))
                    Frame_Portrait_Images.append({
                        "id": [element1['id'][0], element2['id'][0]],
                        "type": "P",
                        "tagLength": len(combined_tags),
                        "tags": combined_tags
                    })
                    Temp_Portrait_Images.pop(j)
                    break

In [5]:
Final_List = Frame_Portrait_Images + Landscape_Images

print(Final_List)

[{'id': [0, 2], 'type': 'P', 'tagLength': 15, 'tags': ['kb6', 'kd7', 'k201', 'kjb1', 'k652', 'k4q', 'k001', 'kg6', 'k17', 'kg11', 'kmz1', 'k8b1', 'k1l', 'k81', 'kw52']}, {'id': [6, 8], 'type': 'P', 'tagLength': 24, 'tags': ['k502', 'kj62', 'k291', 'k1', 'kcd', 'k9s', 'kpz1', 'krx', 'klf', 'k9c1', 'klb', 'k14', 'k911', 'kz9', 'k76', 'kr5', 'kc61', 'kdd1', 'k561', 'km9', 'kk6', 'k3k', 'kl51', 'kr51']}, {'id': [10, 12], 'type': 'P', 'tagLength': 24, 'tags': ['kr02', 'kfk1', 'k582', 'k721', 'kv61', 'kk51', 'kw6', 'kcl', 'kfj', 'kh8', 'k711', 'k692', 'ks1', 'k6k', 'kkb2', 'k0l1', 'k001', 'k3b', 'kb82', 'khd', 'k3d1', 'kwc2', 'kv8', 'ks3']}, {'id': [15, 18], 'type': 'P', 'tagLength': 24, 'tags': ['kc5', 'ksv1', 'klb1', 'kf7', 'kl21', 'kg7', 'kjq', 'k271', 'k452', 'k9n1', 'kdp1', 'kcb2', 'kd7', 'k6d1', 'k2w1', 'k13', 'kd91', 'kz21', 'k82', 'k332', 'kr82', 'kzg1', 'k5r1', 'k572']}, {'id': [20, 26], 'type': 'P', 'tagLength': 27, 'tags': ['kg82', 'kvc2', 'kv1', 'k3g', 'k2m', 'kcl', 'kjr1', 'k112

In [6]:
Tag_List = {}

for i, item in enumerate(Final_List):
    for tag in item['tags']:
        if tag not in Tag_List:
            Tag_List[tag] = {"tag": tag, "id": [i]}
        else:
            Tag_List[tag]["id"].append(i)

# Convert back to the desired list format, if needed
Tag_List = list(Tag_List.values())


In [ ]:
import heapq

# Pre-index Tag_List for fast lookups
tag_index = {tag["tag"]: tag for tag in Tag_List}

Used_images = [0]
Sorted_Images = []
Temp_Sorted_List = Final_List.copy()

if Temp_Sorted_List:
    Sorted_Images.append(Temp_Sorted_List.pop(0))

while Temp_Sorted_List:
    element1 = Sorted_Images[-1]
    all_tag = []

    # Collect all tags associated with element1
    for tag in element1['tags']:
        if tag in tag_index:
            all_tag.append(tag_index[tag])

    # Filter tags associated with more than one image
    target_tag = [t for t in all_tag if len(t['id']) > 1]

    target_images = []

    # Collect and score candidate images
    for tag_data in target_tag:
        for img_id in tag_data['id']:
            if img_id in Used_images:
                continue

            # Check if image is already in target_images
            img_index = next((index for index, d in enumerate(target_images) if d["id"] == img_id), None)
            if img_index is None:
                target_images.append({"id": img_id, "count": 1})
            else:
                target_images[img_index]["count"] += 1

    # Compute minimum tag score for each target image
    for target in target_images:
        target['count'] = min_form_number_of_common_tags(element1, Final_List[target['id']])

    # Use a heap to efficiently find the best candidate
    target_images = [(-t['count'], t['id']) for t in target_images]
    heapq.heapify(target_images)

    if not target_images:
        # Fallback: add the next image from Temp_Sorted_List
        next_image = Temp_Sorted_List.pop(0)
        Used_images.append(next_image["id"])
        Sorted_Images.append(next_image)
    else:
        # Add the best target image
        _, best_image_id = heapq.heappop(target_images)
        Used_images.append(best_image_id)
        Sorted_Images.append(Final_List[best_image_id])
        Temp_Sorted_List.remove(Final_List[best_image_id])


SyntaxError: closing parenthesis ')' does not match opening parenthesis '[' (1835822500.py, line 56)

In [ ]:
def calculate_score(frameglasses):
    """Calculates the Global Robotic Satisfaction."""
    score = 0
    for i in range(len(frameglasses) - 1):
        tags1 = frameglasses[i]
        tags2 = frameglasses[i + 1]
        
        local_score = min_form_number_of_common_tags(tags1, tags2)
        score += local_score

    return score

print(calculate_score(Sorted_Images))

print(Sorted_Images)

0
[]
